In [1]:
#Import the required modules
import pandas as pd
import numpy as np
import io
import sys
import os.path
import urllib.request
from tqdm import tqdm
from os import listdir
from PIL import Image
import glob
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Reading the modified dataset
df = pd.read_csv("Data/data_with_path.csv",encoding='ISO-8859-1')
df.head()

,Title,Release_Year,Genre,Synopsis,Poster_URL,Movie_ID,Movie_URL,Duration,Certification,Voters,Rating,Path
0,John Wick: Chapter 2,(2017),"Action,Crime,Thriller",After returning to the criminal underworld to ...,https://m.media-amazon.com/images/M/MV5BMjE2ND...,4425200,https://www.imdb.com/title/tt4425200/,122 min,A,"438,276",7.4,Posters/4425200.jpg
1,Heartland,(II) (2007â ),"Drama,Family","A multi-generational saga set in Alberta, Cana...",https://m.media-amazon.com/images/M/MV5BY2Q2Zj...,1094229,https://www.imdb.com/title/tt1094229/,45 min,16,"17,385",8.5,Posters/1094229.jpg
2,Don't Look Up,(2021),"Comedy,Drama,Sci-Fi",Two low-level astronomers must go on a giant m...,https://m.media-amazon.com/images/M/MV5BZjcwZj...,11286314,https://www.imdb.com/title/tt11286314/,138 min,18,"541,908",7.2,Posters/11286314.jpg
3,Bosch,(2014â2021),"Crime,Drama",An L.A.P.D. homicide detective works to solve ...,https://m.media-amazon.com/images/M/MV5BZDBiYj...,3502248,https://www.imdb.com/title/tt3502248/,51 min,18,"73,251",8.5,Posters/3502248.jpg
4,The Good Wife,(2009â2016),"Crime,Drama,Mystery",Alicia Florrick has been a good wife to her hu...,https://m.media-amazon.com/images/M/MV5BMTI2OT...,1442462,https://www.imdb.com/title/tt1442462/,43 min,16,"77,212",8.4,Posters/1442462.jpg


In [3]:
# Shape of dataframe
df.shape

(6905, 12)

In [4]:
# There exists a genre value for all images
df['Genre'] = df['Genre'].str.replace(" ", "")
Genre_list = df['Genre']
print(len(Genre_list))

6905


In [5]:
# Breaks "Genre" into the constituting individual genres
def find_genres(genre):
    
    start = 0
    set_of_genre = []
    for i in range(len(genre)):
        
        k=0
        substring = ""
        if (genre[i]==','):
            substring = genre[start:i]
            start = i+1
            k = 1
        
        if(i==len(genre)-1):
            substring = genre[start:i+1]
            k = 1
            
        if (k==1):
            set_of_genre.append(substring)         
    
    return (set_of_genre)

In [6]:
# Extract list of genre values for each image
all_genre = []
Genre_list = df['Genre']

for i in range (len(Genre_list)):
    
    set_of_genre = find_genres(Genre_list[i])
    
    for j in range (len(set_of_genre)):
        all_genre.append(set_of_genre[j])
        
uniq, counts = np.unique(all_genre, return_counts=True)
print("Number of unique genres:", len(uniq))
print("Unique genres are:", uniq)
dict(zip(uniq, counts))

Number of unique genres: 27
Unique genres are: ['A' 'Action' 'Adventure' 'Animation' 'Biography' 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Family' 'Fantasy' 'Game-Show' 'History' 'Horror'
 'Music' 'Musical' 'Mystery' 'News' 'Reality-TV' 'Romance' 'Sci-Fi'
 'Short' 'Sport' 'Talk-Show' 'Thriller' 'War' 'Western']


{'A': 3,
 'Action': 1579,
 'Adventure': 1205,
 'Animation': 492,
 'Biography': 486,
 'Comedy': 2421,
 'Crime': 1379,
 'Documentary': 187,
 'Drama': 3927,
 'Family': 382,
 'Fantasy': 536,
 'Game-Show': 48,
 'History': 250,
 'Horror': 652,
 'Music': 186,
 'Musical': 53,
 'Mystery': 822,
 'News': 6,
 'Reality-TV': 102,
 'Romance': 933,
 'Sci-Fi': 447,
 'Short': 69,
 'Sport': 142,
 'Talk-Show': 13,
 'Thriller': 909,
 'War': 75,
 'Western': 53}

In [7]:
# Reading the dataset
df = pd.read_csv("Data/data_with_path.csv",encoding='ISO-8859-1')

In [8]:
# Prepare multi-hot-encoded-labels for the various genres
def multi_hot_encoded_labels(img_id, genre):
    
    col_names =  ['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
                  'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History',
                  'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance',
                  'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']
    
    set_of_genre = find_genres(genre)
    
    row=[]
    row.append(img_id)
    
    for i in range(len(col_names)):
        
        found = 0
        for j in range (len(set_of_genre)):
            if (set_of_genre[j]==col_names[i]):
                found = 1
                break
        
        row.append(found)
    
    row.append(genre) #add the overall combined genre for record purposes
        
    return row  

In [9]:
# Perform the encoding of the labels and save data in the format :
# Img-ID <multi-hot-encoded-labels> overall_genre
all_data = []

for index, row in tqdm(df.iterrows()):
    
    path = row['Path']
    genre = row['Genre']
    row = multi_hot_encoded_labels(path, genre)
    
    all_data.append(row)

col_names =  ['Img-paths', 'Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
                  'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History',
                  'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance',
                  'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western', 'Genre']

np.savetxt("Data/Multi_hot_encoded_data.csv", np.asarray(all_data), fmt='%s', delimiter=" ")
np.savetxt("Data/Encoded_data_column_lookup.csv", np.asarray(col_names), fmt='%s', delimiter=" ")

6905it [00:01, 5168.03it/s]


In [10]:
# read the previously created data
df_encoded = pd.read_csv("Data/Multi_hot_encoded_data.csv", delimiter=" ", 
                  names =  ['Img-paths', 'Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
                  'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History',
                  'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance',
                  'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western', 'Genre'])

df_encoded.head()

,Img-paths,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,Genre
0,Posters/4425200.jpg,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,"Action,Crime,Thriller"
1,Posters/1094229.jpg,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,"Drama,Family"
2,Posters/11286314.jpg,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,"Comedy,Drama,Sci-Fi"
3,Posters/3502248.jpg,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,"Crime,Drama"
4,Posters/1442462.jpg,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,"Crime,Drama,Mystery"


In [11]:
# To split into train / validation / test in the ratio 80 / 15 / 5%
df = pd.read_csv("Data/Multi_hot_encoded_data.csv", delimiter=" ")
random_seed = 50
train_df = df.sample(frac=0.70, random_state=random_seed) #Taking 70% of the data
tmp_df = df.drop(train_df.index)
test_df = tmp_df.sample(frac=0.1, random_state=random_seed) #Taking 20% of the remaining (after train is taken)
valid_df = tmp_df.drop(test_df.index)

# Display length of data
print("Train_df=",len(train_df))
print("Val_df=",len(valid_df))
print("Test_df=",len(test_df))

# Save the data into dataset
np.savetxt("Data/Train.csv", train_df, fmt='%s', delimiter=" ")
np.savetxt("Data/Test.csv", test_df, fmt='%s', delimiter=" ")
np.savetxt("Data/Valid.csv", valid_df, fmt='%s', delimiter=" ")

Train_df= 4833
Val_df= 1864
Test_df= 207
